<a href="https://colab.research.google.com/github/SiddharthVikram068/IndusChain/blob/main/manufacturing_ui/predictive_model/deploy/deployment_failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install joblib
!pip install modelbit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: build
    Found existing installation: build 1.2.2.post1
    Uninstalling build-1.2.2.post1:
      Successfully uninstalled build-1.2.2.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pip-tools 7.4.1 requires build>=1.0.0, but you have build 0.10.0 which is incompatible.


In [15]:
import modelbit
mb = modelbit.login()

In [18]:
import numpy as np

In [2]:
import joblib


In [4]:
rfc = joblib.load('model_xgboost.joblib')

In [ ]:
def get_user_input():
    # Getting the input data from the user
    print("Enter the input parameters for the predictive maintenance classification:")

    selected_type = input("Select a Type (Low/Medium/High) with L/M/H: ")
    type_mapping = {'L': 0, 'M': 1, 'H': 2}
    if selected_type not in type_mapping:
        print("Invalid selection. Please choose from Low, Medium, or High.")
        return None

    air_temperature = float(input('Air temperature [K]: '))
    process_temperature = float(input('Process temperature [K]: '))
    rotational_speed = float(input('Rotational speed [rpm]: '))
    torque = float(input('Torque [Nm]: '))
    tool_wear = float(input('Tool wear [min]: '))

    return [type_mapping[selected_type], air_temperature, process_temperature,
            rotational_speed, torque, tool_wear]


In [38]:
sample_user_input = ['M', 298.15, 305.15, 1500, 50, 200]

In [26]:
type_mapping = {'L': 0, 'M': 1, 'H': 2}
sample_user_input[0] = type_mapping[sample_user_input[0]]

In [27]:
user_input = sample_user_input



    # Predicting failure
failure_pred = rfc.predict([user_input])

if failure_pred[0] == 1:
    print("Prediction: Failure")
else:
    print("Prediction: No Failure")


Prediction: No Failure


In [19]:
def predict_failure(input_data):
    # Here you would convert input_data to the proper format
    # Example assuming input_data is already in the right format:
    user_input = np.array([input_data])

    # Predict failure
    failure_pred = rfc.predict(user_input)

    if failure_pred[0] == 1:
        return "Prediction: Failure"
    else:
        return "Prediction: No Failure"

In [32]:
def predict_failure_dp(selected_type, air_temperature, process_temperature, rotational_speed, torque, tool_wear):
    # Convert the categorical type input into its numerical mapping
    type_mapping = {'L': 0, 'M': 1, 'H': 2}
    type_value = type_mapping.get(selected_type, -1)

    if type_value == -1:
        return "Error: Invalid Type"

    # Create the input array
    user_input = np.array([[type_value, air_temperature, process_temperature, rotational_speed, torque, tool_wear]])

    # Predict failure
    failure_pred = rfc.predict(user_input)

    if failure_pred[0] == 1:
        return "Prediction: Failure"
    else:
        return "Prediction: No Failure"

In [36]:
predict_failure_dp('M', 298.15, 305.15, 1500, 50, 200)

'Prediction: No Failure'

In [33]:
mb.deploy(predict_failure_dp)

In [39]:
result =mb.get_inference(
  region="us-east-2.aws",
  workspace="aryamanpathak",
  deployment="predict_failure_dp",
  data=sample_user_input
)
print(result)

print([user_input])

{'data': 'Prediction: No Failure'}
[[1, 298.15, 305.15, 1500, 50, 200]]
